In [1]:
# 참고 : https://github.com/yojulab/learn_data_analytics/wiki/10.3--CDA
# 참고(scipy api) : https://docs.scipy.org/doc/scipy/reference/stats.html#module-scipy.stats

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font',family='Malgun Gothic')
mpl.rcParams['axes.unicode_minus'] = False
import seaborn as sns

In [3]:
df_TOCC=pd.read_csv("../../datasets/UsingChart_TypeOfContractChannel.csv")
df_TOCC[:3]

,id,type_of_contract,type_of_contract2,channel,datetime,Term,payment_type,product,amount,state,overdue_count,overdue,credit rating,bank,cancellation,age,Mileage
0,66758234,렌탈,Normal,서비스 방문,2019-10-20,60,CMS,K1,96900,계약확정,0,없음,9.0,새마을금고,정상,43.0,1862.0
1,66755948,렌탈,Extension_Rental,서비스 방문,2019-10-20,60,카드이체,K1,102900,계약확정,0,없음,2.0,현대카드,정상,62.0,2532.0
2,66756657,렌탈,Normal,홈쇼핑/방송,2019-10-20,60,CMS,K1,96900,계약확정,0,없음,8.0,우리은행,정상,60.0,2363.0


### x 범주형,  y 범주형 : 주로 독립성 증명

#### 선정 변수 : type_of_contract, cancellation
- 분석 내용 : 계약유형에 따라 해약여부가 달라지는가?
    - 귀무가설 : 계약유형에 따라 해약여부가 달라지지 않는다.
    - 대립가설 : 계약유형에 따라 해약여부가 달라진다.
- 분석 결론 (기준 : p-value 0.05)
    - 통계 결론 : 대립가설 참, 
    - 사실 결론 : 계약유형에 따라 해약여부가 달라진다.

In [4]:
df_TOCC[['type_of_contract', 'cancellation']].describe()

,type_of_contract,cancellation
count,51300,51279
unique,2,2
top,렌탈,정상
freq,46481,50372


In [5]:
df_TOCC_category = df_TOCC[['type_of_contract', 'cancellation']]

In [6]:
df_TOCC_category_pivot = pd.pivot_table(data=df_TOCC_category, index='type_of_contract', columns='cancellation', aggfunc=len)
df_TOCC_category_pivot

cancellation,정상,해약
type_of_contract,,
렌탈,46137,344
멤버십,4231,563


In [7]:
import scipy 
scipy.__version__
import scipy.stats as stats

In [8]:
stats.chi2_contingency(observed=df_TOCC_category_pivot, correction=True)

Chi2ContingencyResult(statistic=3021.9735309482894, pvalue=0.0, dof=1, expected_freq=array([[45658.8007411,   822.1992589],
       [ 4709.1992589,    84.8007411]]))

In [11]:
df_TOCC[['state', 'cancellation']].describe()

,state,cancellation
count,51304,51279
unique,4,2
top,계약확정,정상
freq,50350,50372


#### 선정 변수 : product, cancellation
- 분석 내용 : 차종에 따라 해약여부가 달라지는가?
    - 귀무가설 : 차종에 따라 해약여부가 달라지지 않는다.
    - 대립가설 : 차종에 따라 해약여부가 달라진다.
- 분석 결론 (기준 : p-value 0.05)
    - 통계 결론 : 대립가설 참
    - 사실 결론 : 차종에 따라 해약여부가 달라진다.

In [17]:
# 다른변수이름 작성 시 copy() 사용
df_TOCC_product = df_TOCC[['product', 'cancellation']].dropna().copy()
# df_TOCC_product[:3]
df_TOCC_product.describe()

,product,cancellation
count,51278,51278
unique,6,2
top,K1,정상
freq,39111,50371


In [29]:
df_TOCC_product_pivot = pd.pivot_table(data=df_TOCC_product, index='product'
                                       , columns='cancellation', aggfunc=len).copy()
# df_TOCC_product_pivot

In [30]:
## product 기준
stats.chi2_contingency(observed=df_TOCC_product_pivot, correction=True)

Chi2ContingencyResult(statistic=103.79474317337241, pvalue=8.372028115481391e-21, dof=5, expected_freq=array([[3.84192086e+04, 6.91791353e+02],
       [8.83393274e+03, 1.59067261e+02],
       [2.04517380e+03, 3.68262023e+01],
       [3.21216058e+02, 5.78394243e+00],
       [6.33591306e+02, 1.14086938e+01],
       [1.17877452e+02, 2.12254768e+00]]))

In [28]:
## correction 기준
stats.chi2_contingency(observed=df_TOCC_product_pivot, correction=True)

Chi2ContingencyResult(statistic=103.79474317337241, pvalue=8.372028115481391e-21, dof=5, expected_freq=array([[3.84192086e+04, 8.83393274e+03, 2.04517380e+03, 3.21216058e+02,
        6.33591306e+02, 1.17877452e+02],
       [6.91791353e+02, 1.59067261e+02, 3.68262023e+01, 5.78394243e+00,
        1.14086938e+01, 2.12254768e+00]]))

#### 선정 변수 : state, cancellation
- 이런 변수 구성은 참조되지 않는다.

In [24]:
# error with Non
df_TOCC_error = df_TOCC[['state', 'cancellation']].dropna().copy()
# df_TOCC_error[:3]
df_TOCC_error.describe()

,state,cancellation
count,51279,51279
unique,4,2
top,계약확정,정상
freq,50326,50372


In [25]:
df_TOCC_error_pivot = pd.pivot_table(data=df_TOCC_error, index='state'
               , columns='cancellation', aggfunc=len)
df_TOCC_error_pivot

cancellation,정상,해약
state,,
계약확정,50326.0,NaN
기간만료,32.0,NaN
해약진행중,14.0,NaN
해약확정,NaN,907.0


In [26]:
stats.chi2_contingency(observed=df_TOCC_error_pivot, correction=True)

Chi2ContingencyResult(statistic=nan, pvalue=nan, dof=3, expected_freq=array([[nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan]]))